In [1]:
import os, time, glob, socket
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import csv
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


In [2]:
os.getcwd() # 現在的路徑
path=os.getcwd()

# 1.讀取data

In [3]:
X_train = pd.read_csv("X_train.csv",index_col=0) #不重新給 Unnamed
y_train = pd.read_csv("y_train.csv",index_col=0) #不重新給 Unnamed
X_test = pd.read_csv("X_test.csv",index_col=0) #不重新給 Unnamed
y_test = pd.read_csv("X_test.csv",index_col=0) #不重新給 Unnamed
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1000, 150)
(500, 150)
(1000, 2)
(500, 150)


# 2.先把 X_train 、 X_test 上下合併


In [4]:
# axis=0 上下合併
# ignore_index=True 可以忽略合併時舊的 index 欄位，改採用自動產生的 index
X_all=pd.concat([X_train,X_test],axis=0,ignore_index=True) 

# 3. 先把missing value太多的變數刪除

In [5]:
# 看 MISSING VALUE 

print(len(X_all.columns))
X_all_isnull=X_all.isnull().sum(axis=0)
X_all_with_na_le20=X_all_isnull[X_all_isnull < 0.2*X_all.shape[0]]## 刪除 有 >20% missing value 的變數 ---
X_all=X_all[X_all_with_na_le20.index] 
print(len(X_all.columns))


150
92


# 3. 變數分類(連續cont、有序類別order、無序類別cate)

In [6]:
X_all.dtypes

Unnamed: 0.1                    int64
id                              int64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 92, dtype: object

In [7]:
# fuction

def cate_cont(X_all):
    
    cate = [] # object
    cont = [] # int64、float64

    for i in range(X_all.shape[1]):
        if X_all.dtypes[i] == "object":
            cate.append(X_all.columns[i])
        else:
            cont.append(X_all.columns[i])
    return cont , cate

In [8]:
print(cate_cont(X_all)[1]) # object
print(len(cate_cont(X_all)[1]))

['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type', 'hardship_flag', 'debt_settlement_flag']
23


### 3.1 處理 object 的變數

#####  % 轉小數 (有 NA 沒關系)

In [9]:
list1=['int_rate','revol_util'] #要轉的變數

for i in range(0,len(list1)):
       X_all[list1[i]]= X_all[list1[i]].str.rstrip('%').astype('float') / 100.0
        

##### 拿掉單位 (如果有 NA--可能會沒辦法刪除 )

In [10]:
list2=['term'] #要轉的變數 可能要一個一個改 因為單位長度不一樣


for i in range(0,len(list2)):

    print(X_all[list2[i]].unique())

    X_all[list2[i]] =X_all[list2[i]].astype(str)            # 將 nan 一併轉成 str
    X_all[list2[i]]=X_all[list2[i]].apply(lambda x:x[0:-6]) # #從後面刪除數個字母 #nan可能全部被刪除剩下空格

    X_all[list2[i]] = X_all[list2[i]].replace('', np.nan)   # 把空格填回 np.nan
    X_all[list2[i]] =X_all[list2[i]].astype('float64')


[nan ' 36 months' ' 60 months']


##### 改掉特殊符號 (有 NA 沒關系)

In [11]:
list3=['emp_length'] #要轉的變數 可能要一個一個改 因為單位長度不一樣

for i in range(0,len(list3)):

    print(X_all[list3[i]].unique())
    X_all[list3[i]]=X_all[list3[i]].str.replace('<','less than')
    X_all[list3[i]]=X_all[list3[i]].str.replace('+','more than')

    print(X_all[list3[i]].unique())

['1 year' '10+ years' '2 years' '< 1 year' '4 years' '3 years' '5 years'
 '6 years' '7 years' nan '8 years' '9 years']
['1 year' '10more than years' '2 years' 'less than 1 year' '4 years'
 '3 years' '5 years' '6 years' '7 years' nan '8 years' '9 years']


In [12]:
# 處理日期 issue_d ......



### 3.2 決定object要留下的變數--判斷哪些是category 哪些是order  哪些num

In [12]:
obj=cate_cont(X_all)[1] # object
obj.sort(reverse=False)
print(obj)

['addr_state', 'application_type', 'debt_settlement_flag', 'earliest_cr_line', 'emp_length', 'emp_title', 'grade', 'hardship_flag', 'home_ownership', 'initial_list_status', 'issue_d', 'last_credit_pull_d', 'last_pymnt_d', 'purpose', 'pymnt_plan', 'sub_grade', 'title', 'url', 'verification_status', 'zip_code']


In [13]:
# 選擇要留的
# 要留 'Unnamed: 0.1' 

order=['Unnamed: 0.1']+['grade','sub_grade']

In [14]:
# 選擇要留的
# 要留 'Unnamed: 0.1' 

cate=['Unnamed: 0.1']+['application_type', 'debt_settlement_flag', 'emp_length', 
                       'emp_title', 'hardship_flag', 'home_ownership', 'initial_list_status', 'purpose', 'pymnt_plan', 'sub_grade', 'title', 
                       'verification_status']
print(cate)

['Unnamed: 0.1', 'application_type', 'debt_settlement_flag', 'emp_length', 'emp_title', 'hardship_flag', 'home_ownership', 'initial_list_status', 'purpose', 'pymnt_plan', 'sub_grade', 'title', 'verification_status']


In [15]:
# 選擇要留的
# 要留 'Unnamed: 0.1' 

num=cate_cont(X_all)[0]
num.remove('id')
print(num)

['Unnamed: 0.1', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_r

# 4.分別補值、轉換

In [16]:
X_num=X_all[num]
X_cate=X_all[cate]
X_order=X_all[order]
print(X_num.shape)
print(X_cate.shape)
print(X_order.shape)

(1500, 71)
(1500, 13)
(1500, 3)


## cate

### 4.1 cate 把 na當成一類

In [17]:
X_cate.isnull().sum(axis=0)

Unnamed: 0.1              0
application_type          0
debt_settlement_flag      0
emp_length              108
emp_title               131
hardship_flag            36
home_ownership            0
initial_list_status       0
purpose                   0
pymnt_plan                0
sub_grade                 0
title                    14
verification_status       0
dtype: int64

In [18]:
X_cate=X_cate.replace(np.nan,'unknown')

In [19]:
X_cate.isnull().sum(axis=0)

Unnamed: 0.1            0
application_type        0
debt_settlement_flag    0
emp_length              0
emp_title               0
hardship_flag           0
home_ownership          0
initial_list_status     0
purpose                 0
pymnt_plan              0
sub_grade               0
title                   0
verification_status     0
dtype: int64

### 4.2 轉成 dummy

In [20]:
X_cate_dum = pd.get_dummies(X_cate)
print(X_cate_dum.shape)

(1500, 1150)


In [21]:
X_cate_dum

,Unnamed: 0.1,application_type_Individual,application_type_Joint App,debt_settlement_flag_N,debt_settlement_flag_Y,emp_length_1 year,emp_length_10more than years,emp_length_2 years,emp_length_3 years,emp_length_4 years,...,title_Moving and relocation,title_Other,title_Vacation,title_baseball,title_new beginning,title_pay off rip off loan,title_unknown,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,99620,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1676459,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1192524,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2230019,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,452696,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1101193,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1496,2419501,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1497,1243704,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1498,526586,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## order

### 4.3 將nan用mode補植

In [22]:
X_order.isnull().sum(axis=0)

Unnamed: 0.1    0
grade           0
sub_grade       0
dtype: int64

In [23]:
var=X_order.columns.tolist()

for i in range(0,len(var)): 
     X_order[str(var[i])]=X_order[str(var[i])].fillna(X_order[str(var[i])].mode()[0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
X_order.isnull().sum(axis=0)

Unnamed: 0.1    0
grade           0
sub_grade       0
dtype: int64

### 4.4 轉成有序的dummy

In [25]:
print(X_order['grade'].unique())

['A' 'B' 'C' 'D' 'E' 'G' 'F']


In [26]:
# 方法一

var=X_order.columns.tolist()
X_order_dummy=pd.DataFrame(columns=X_order.columns)

for i in range(0,len(var)): 

    le = LabelEncoder()
    le.fit(X_order[str(var[i])].unique())  
    X_order_dummy[str(var[i])]=le.transform(X_order[str(var[i])])

In [27]:
# 方法二

# Dict = {'A':0 ,'B': 1,'C': 2,'D': 3,'E': 4,'F':5,'G':6}
# X_order['grade'] = X_order['grade'].map(Dict)

In [28]:
print(X_order_dummy.shape)

(1500, 3)


## num

### 方法一: 用 mean補值

In [29]:
X_num=X_all[num]
X_num.isnull().sum(axis=0)

Unnamed: 0.1                  0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          4
                             ..
tax_liens                     0
tot_hi_cred_lim               0
total_bal_ex_mort             0
total_bc_limit                0
total_il_high_credit_limit    0
Length: 71, dtype: int64

In [30]:
var=X_num.columns.tolist()
X_num_imp_mean=pd.DataFrame(columns=X_num.columns)

for i in range(0,len(var)): 
     X_num_imp_mean[str(var[i])]=X_num[str(var[i])].fillna(X_num[str(var[i])].mean())

print(X_num_imp_mean.shape)

(1500, 71)


In [31]:
X_num_imp_mean.isnull().sum(axis=0)

Unnamed: 0.1                  0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
                             ..
tax_liens                     0
tot_hi_cred_lim               0
total_bal_ex_mort             0
total_bc_limit                0
total_il_high_credit_limit    0
Length: 71, dtype: int64

### 方法二: 用 knn補值

In [32]:
X_num=X_all[num]
X_num.isnull().sum(axis=0)

Unnamed: 0.1                  0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          4
                             ..
tax_liens                     0
tot_hi_cred_lim               0
total_bal_ex_mort             0
total_bc_limit                0
total_il_high_credit_limit    0
Length: 71, dtype: int64

In [ ]:
pip install fancyimpute

In [ ]:
import fancyimpute
from fancyimpute import KNN
import random

X_num_imp_knn = pd.DataFrame(KNN(k=5).fit_transform(X_num),columns = X_num.columns)


### 方法三: 用 mice補值

In [ ]:
from sklearn.impute import IterativeImputer

In [ ]:
# 需要補值的整筆 data
use_var=['id']+num_without_na.index.tolist()+num_with_na_le20.index.tolist() #加入 id
X_num_use_var=X_all[use_var]
print(X_num_use_var.shape)

# data中沒有 missing value 的部分(觀測值)
filter=X_num_use_var.isnull().sum(axis=1)==0
X_num_use_var_rowwithoutna=X_num_use_var[filter] # 含有 NA的 觀測值分
print(X_num_use_var_rowwithoutna.shape)

# data中有 missing value 的部分(觀測值)
filter=X_num_use_var.isnull().sum(axis=1)!=0
X_num_use_var_rowwithna=X_num_use_var[filter] # 含有 NA的 觀測值分
print(X_num_use_var_rowwithna.shape)

In [ ]:
# 補植 IterativeImputer
imp = IterativeImputer(max_iter=20, random_state=0)
imp.fit(X_num_use_var) # 整筆 _X_train_num
                                        
                                                   # 要補值的部分
X_num_rowwithna_imp=pd.DataFrame(imp.transform(X_num_use_var_rowwithna),index=X_num_use_var_rowwithna.index,columns=X_num_use_var_rowwithna.columns)
print(X_num_rowwithna_imp.shape)


# 上下合併  補完後的部分 和 沒有 missing value(觀測直) 的部分合併
X_num_imp=pd.concat([X_num_use_var_rowwithoutna,X_num_rowwithna_imp],axis=0)
# 確保每個觀測直的位子和原本一樣，沒有打亂。前半是train 
X_num_imp=X_num_imp.sort_index()  
print(X_num_imp.shape)